## DP-1 Oliver Adler demo

Nakoniec som sa rozhodol pre hľadanie implicitných vzťahov medzi vzormi. Používal som zatiaľ iba organizačné vzory.

In [46]:
import os
import pandas as pd
import numpy as np
from itertools import product, combinations
import transformers
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
from transformers import LongformerTokenizerFast, \
LongformerModel, LongformerTokenizer, LongformerConfig, Trainer, TrainingArguments, EvalPrediction, AutoTokenizer, AutoModel
from transformers.models.longformer.modeling_longformer import LongformerPreTrainedModel, LongformerClassificationHead

from tensorflow.keras import layers
from tensorflow.keras import losses

Pre toto demo som použil vzory z jazyka ProjectManagementPatternLanguage a PiecemealGrowthPatternLanguage. Z každého vzoru vezmem jeho názov, počiatočný kontext (= problém ktorý rieši) a jeho konečný kontext. Text rozdeľujem podľa schémy, ktorú vzory z tejto kolekcie majú. 

In [134]:
directory = "patterns/ProjectManagmentPatternLanguage"
starting_contexts = []
resulting_contexts = []
pattern_names = []
pattern_languages = []
for filename in os.listdir(directory):
    print(filename)
    with open(directory+"/"+filename) as f:
        lines = f.readlines()
        pattern_names.append(lines[0].replace('\n',''))
        pattern_languages.append("ProjectManagmentPatternLanguage")
        starting_context = ""
        
        resulting_context = ""
        starting_context_flag = False
        resulting_context_flag = False
        for line in lines:
            #end of resulting context statement
            if "âœ¥" in line and resulting_context_flag:
                break
            
            #beginning of starting context statement
            if "âœ¥" in line and not starting_context_flag:
                starting_context_flag = True
                continue
                
            #beginning of resulting context statement
            if "Therefore:" in line and starting_context_flag:
                starting_context_flag = False
                resulting_context_flag = True
                continue

            if starting_context_flag and line != "\n":
                starting_context+=line

            if resulting_context_flag and line != "\n":
                resulting_context+=line
      
        starting_contexts.append(starting_context.translate({ord('\n'): None}))
        resulting_contexts.append(resulting_context.translate({ord('\n'): None}))
directory = "patterns/PiecemealGrowthPatternLanguage"
for filename in os.listdir(directory):
    with open(directory+"/"+filename) as f:
        lines = f.readlines()
        pattern_names.append(lines[0].replace('\n',''))
        pattern_languages.append("PiecemealGrowthPatternLanguage")
        starting_context = ""
        
        resulting_context = ""
        starting_context_flag = False
        resulting_context_flag = False
        for line in lines:
            #end of resulting context statement
            if "âœ¥" in line and resulting_context_flag:
                break
            
            #beginning of starting context statement
            if "âœ¥" in line and not starting_context_flag:
                starting_context_flag = True
                continue
                
            #beginning of resulting context statement
            if "Therefore:" in line and starting_context_flag:
                starting_context_flag = False
                resulting_context_flag = True
                continue

            if starting_context_flag and line != "\n":
                starting_context+=line

            if resulting_context_flag and line != "\n":
                resulting_context+=line
      
        starting_contexts.append(starting_context.translate({ord('\n'): None}))
        resulting_contexts.append(resulting_context.translate({ord('\n'): None}))

BuildPrototypes.txt
CommunityOfTrust.txt
CompletionHeadroom.txt
DayCare.txt
DeveloperControlsProcess.txt
DevelopmentEpisode.txt
DontInteruptAnInterrupt.txt
GetOnWithIt.txt
ImpliedRequirements.txt
IncrementalIntegration.txt
InformalLaborPlan.txt
InterruptsUnjamBlocking.txt
MercenaryAnalyst.txt
NamedStableBases.txt
PrivateWorld.txt
ProgrammingEpisode.txt
RecommitmentMeeting.txt
SacrificeOnePerson.txt
SizeTheSchedule.txt
SomeoneAlwaysMakesProgress.txt
TakeNoSmallSlips.txt
TeamPerTask.txt
WorkFlowsInward.txt
WorkQueue.txt
WorkSplit.txt


In [135]:
patterns=[]
for i,pattern_name in enumerate(pattern_names):
    patterns.append(pattern_name)
    patterns.append(starting_contexts[i])
    patterns.append(resulting_contexts[i])

In [136]:
d = {"pattern_names": pattern_names, 'pattern_language': pattern_languages,'starting_contexts': starting_contexts, "resulting_contexts": resulting_contexts}
df = pd.DataFrame(data=d)

In [137]:
df

,pattern_names,pattern_language,starting_contexts,resulting_contexts
0,Build Prototypes,ProjectManagmentPatternLanguage,A project must test requirements and design de...,Build an isolated prototype solution whose pur...
1,Community Of Trust,ProjectManagmentPatternLanguage,It is essential that the people in a team trus...,"Do things that explicitly demonstrate trust, s..."
2,Completion Headroom,ProjectManagmentPatternLanguage,Every project must commit to delivery on a few...,Project work group completion dates from remai...
3,Day Care,ProjectManagmentPatternLanguage,Your experts are spending all their time mento...,"Put one expert in charge of all the novices, l..."
4,Developer Controls Process,ProjectManagmentPatternLanguage,"A development culture, like any culture, can b...",Make the Developer the focal point of process ...
5,Development Episode,ProjectManagmentPatternLanguage,It's important to build on the collective stre...,Approach all development as a group activity a...
6,Don't Interrupt An Interrupt,ProjectManagmentPatternLanguage,It's important to balance a desire that Someon...,"If a developer is already working in ""interrup..."
7,Get On With It,ProjectManagmentPatternLanguage,You can't wait until you have every last requi...,As soon as you have confidence about some proj...
8,Implied Requirements,ProjectManagmentPatternLanguage,A commitment implies an agreement between peop...,Select and name chunks of functionality. Use n...
9,Incremental Integration,ProjectManagmentPatternLanguage,"For iterative development to work well, it is ...",Provide a mechanism to allow developers to bui...


Zo vzorov vytváram dvojice, kde pre dvojicu vzorov X a Y, budem mať v dataframe ich názvy, počiatočný kontext vzoru X a konečný kontext vzoru Y 

In [138]:
pattern_pairs = list(combinations(df['pattern_names'].tolist(),2))
pattern_pairs

[('Build Prototypes', 'Community Of Trust'),
 ('Build Prototypes', 'Completion Headroom'),
 ('Build Prototypes', 'Day Care'),
 ('Build Prototypes', 'Developer Controls Process'),
 ('Build Prototypes', 'Development Episode'),
 ('Build Prototypes', "Don't Interrupt An Interrupt"),
 ('Build Prototypes', 'Get On With It'),
 ('Build Prototypes', 'Implied Requirements'),
 ('Build Prototypes', 'Incremental Integration'),
 ('Build Prototypes', 'Informal Labor Plan'),
 ('Build Prototypes', 'Interrupts Unjam Blocking'),
 ('Build Prototypes', 'Mercenary Analyst'),
 ('Build Prototypes', 'Named Stable Bases'),
 ('Build Prototypes', 'Private World'),
 ('Build Prototypes', 'Programming Episode'),
 ('Build Prototypes', 'Recommitment Meeting '),
 ('Build Prototypes', 'Sacrifice One Person '),
 ('Build Prototypes', 'Size The Schedule'),
 ('Build Prototypes', 'Someone Always Makes Progress '),
 ('Build Prototypes', 'Take No Small Slips'),
 ('Build Prototypes', 'Team Per Task'),
 ('Build Prototypes', 'Wor

In [139]:
df_new = pd.DataFrame(list(combinations(df['pattern_names'].tolist(),2)), columns=['pattern_name1','pattern_name2'])
df_new

,pattern_name1,pattern_name2
0,Build Prototypes,Community Of Trust
1,Build Prototypes,Completion Headroom
2,Build Prototypes,Day Care
3,Build Prototypes,Developer Controls Process
4,Build Prototypes,Development Episode
...,...,...
1426,urrogate Customer,Unity Of Purpose
1427,urrogate Customer,Wise Fool
1428,Team Pride,Unity Of Purpose
1429,Team Pride,Wise Fool


In [140]:
df_new["resulting_context1"] = df_new.apply(lambda row: df[df["pattern_names"]==str(row.pattern_name1)].resulting_contexts.item(), axis=1)
df_new["resulting_context2"] = df_new.apply(lambda row: df[df["pattern_names"]==str(row.pattern_name2)].resulting_contexts.item(), axis=1)
df_new["starting_context1"] = df_new.apply(lambda row: df[df["pattern_names"]==str(row.pattern_name1)].starting_contexts.item(), axis=1)
df_new["starting_context2"] = df_new.apply(lambda row: df[df["pattern_names"]==str(row.pattern_name2)].starting_contexts.item(), axis=1)
df_new

,pattern_name1,pattern_name2,resulting_context1,resulting_context2,starting_context1,starting_context2
0,Build Prototypes,Community Of Trust,Build an isolated prototype solution whose pur...,"Do things that explicitly demonstrate trust, s...",A project must test requirements and design de...,It is essential that the people in a team trus...
1,Build Prototypes,Completion Headroom,Build an isolated prototype solution whose pur...,Project work group completion dates from remai...,A project must test requirements and design de...,Every project must commit to delivery on a few...
2,Build Prototypes,Day Care,Build an isolated prototype solution whose pur...,"Put one expert in charge of all the novices, l...",A project must test requirements and design de...,Your experts are spending all their time mento...
3,Build Prototypes,Developer Controls Process,Build an isolated prototype solution whose pur...,Make the Developer the focal point of process ...,A project must test requirements and design de...,"A development culture, like any culture, can b..."
4,Build Prototypes,Development Episode,Build an isolated prototype solution whose pur...,Approach all development as a group activity a...,A project must test requirements and design de...,It's important to build on the collective stre...
...,...,...,...,...,...,...
1426,urrogate Customer,Unity Of Purpose,Create a SurrogateCustomer role in the project...,The leader of the project must instill a commo...,It is important to exchange ideas and clarify ...,Many projects have rocky beginnings as people ...
1427,urrogate Customer,Wise Fool,Create a SurrogateCustomer role in the project...,"Nurture the role of the wise fool, who can rai...",It is important to exchange ideas and clarify ...,Interpersonal dynamics often discourage good i...
1428,Team Pride,Unity Of Purpose,"""We're the best."" Instill a sense of elitism i...",The leader of the project must instill a commo...,People are most successful when they feel good...,Many projects have rocky beginnings as people ...
1429,Team Pride,Wise Fool,"""We're the best."" Instill a sense of elitism i...","Nurture the role of the wise fool, who can rai...",People are most successful when they feel good...,Interpersonal dynamics often discourage good i...


## Skladanie vzorov do sekvencií (nápad)

V tomto bode som skúsil zo sekvencie opísanej v jazyku vzorov ProjectManagementPatternLanguage 
(A Story About Project Management - https://sites.google.com/a/scrumplop.org/published-patterns/Organizational-Patterns-of-Agile-Software-Development/bookoutline/thepatternlanguages/organizationdesignpatterns/projectmanagementpatternlanguage?authuser=0). 


Z príbehu som vybral postupnosti vzorov tak ako išli za sebou. Napríklad prvý je spomínaný vzor BuildPrototypes a následne bol aplikovaný SizeTheSchedule. Takže do tabuľky spravím záznam sequence==1 pre záznam kde sa nachádza tento pár. Keďže som sa chcel zamerať na to kedy, respektíve po aplikovaní akého vzrou je môžné/vhodné aplikovať druhý vzor, bral som tento vzťah len pre dvojicu po sebe idúcich vzorov v príbehu. Teda, po SizeTheSchedule nasleduje vzor NamedStableBases. Dvojica SizeTheSchedule a NamedStableBases budú mať sequence==1, ale pár BuildPrototypes a NamedStableBases bude mať 0 (Nemusí to byť správne uvažovanie). 

In [364]:
df_final["sequence"]=0
df_final

C:\Users\olive\AppData\Local\Temp\ipykernel_4724\4076506384.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["sequence"]=0


,pattern_name1,pattern_name2,resulting_context,starting_context,sequence
1,Build Prototypes,Community Of Trust,Build an isolated prototype solution whose pur...,It is essential that the people in a team trus...,0
2,Build Prototypes,Completion Headroom,Build an isolated prototype solution whose pur...,Every project must commit to delivery on a few...,0
3,Build Prototypes,Day Care,Build an isolated prototype solution whose pur...,Your experts are spending all their time mento...,0
4,Build Prototypes,Developer Controls Process,Build an isolated prototype solution whose pur...,"A development culture, like any culture, can b...",0
5,Build Prototypes,Development Episode,Build an isolated prototype solution whose pur...,It's important to build on the collective stre...,0
...,...,...,...,...,...
670,Work Split,Surrogate Customer,Divide a task into an urgent and deferred comp...,It is important to exchange ideas and clarify ...,0
671,Work Split,Take No Small Slips,Divide a task into an urgent and deferred comp...,It's difficult to know how long a project shou...,0
672,Work Split,Team Per Task,Divide a task into an urgent and deferred comp...,Large distractions (usually called crises) mus...,0
673,Work Split,Work Flows Inward,Divide a task into an urgent and deferred comp...,An organization must seek a structure that bes...,0


In [365]:
#vytvorené dvojice z príbehu
indexes = []

indexes.append(df_final.index[(df_final.pattern_name1=="Build Prototypes") & (df_final.pattern_name2=="Size The Schedule")])
indexes.append(df_final.index[(df_final.pattern_name1=="Size The Schedule") & (df_final.pattern_name2=="Named Stable Bases")])
indexes.append(df_final.index[(df_final.pattern_name1=="Named Stable Bases") & (df_final.pattern_name2=="Private World")])
indexes.append(df_final.index[(df_final.pattern_name1=="Private World") & (df_final.pattern_name2=="Developer Controls Process")])
indexes.append(df_final.index[(df_final.pattern_name1=="Developer Controls Process") & (df_final.pattern_name2=="Work Flows Inward")])
indexes.append(df_final.index[(df_final.pattern_name1=="Developer Controls Process") & (df_final.pattern_name2=="Work Queue")])
indexes.append(df_final.index[(df_final.pattern_name1=="Developer Controls Process") & (df_final.pattern_name2=="Informal Labor Plan")])
indexes.append(df_final.index[(df_final.pattern_name1=="Developer Controls Process") & (df_final.pattern_name2=="Programming Episode")])
indexes.append(df_final.index[(df_final.pattern_name1=="Work Queue") & (df_final.pattern_name2=="Take No Small Slips")])
indexes.append(df_final.index[(df_final.pattern_name1=="Informal Labor Plan") & (df_final.pattern_name2=="Take No Small Slips")])
indexes.append(df_final.index[(df_final.pattern_name1=="Programming Episode") & (df_final.pattern_name2=="Take No Small Slips")])
indexes.append(df_final.index[(df_final.pattern_name1=="Take No Small Slips") & (df_final.pattern_name2=="Recommitment Meeting ")])
indexes.append(df_final.index[(df_final.pattern_name1=="Take No Small Slips") & (df_final.pattern_name2=="Team Per Task")])
indexes.append(df_final.index[(df_final.pattern_name1=="Take No Small Slips") & (df_final.pattern_name2=="Sacrifice One Person ")])
indexes.append(df_final.index[(df_final.pattern_name1=="Team Per Task") & (df_final.pattern_name2=="Someone Always Makes Progress ")])

indexes

[Int64Index([18], dtype='int64'),
 Int64Index([481], dtype='int64'),
 Int64Index([352], dtype='int64'),
 Int64Index([368], dtype='int64'),
 Int64Index([127], dtype='int64'),
 Int64Index([128], dtype='int64'),
 Int64Index([114], dtype='int64'),
 Int64Index([119], dtype='int64'),
 Int64Index([645], dtype='int64'),
 Int64Index([281], dtype='int64'),
 Int64Index([411], dtype='int64'),
 Int64Index([562], dtype='int64'),
 Int64Index([568], dtype='int64'),
 Int64Index([563], dtype='int64'),
 Int64Index([591], dtype='int64')]

In [366]:
for index in indexes:
    df_final.loc[index,'sequence']=1
df_final[df_final['sequence']==1]

,pattern_name1,pattern_name2,resulting_context,starting_context,sequence
18,Build Prototypes,Size The Schedule,Build an isolated prototype solution whose pur...,Both overly ambitious schedules and overly gen...,1
114,Developer Controls Process,Informal Labor Plan,Make the Developer the focal point of process ...,A schedule of developer work tasks can both as...,1
119,Developer Controls Process,Programming Episode,Make the Developer the focal point of process ...,Programming is the act of deciding now what wi...,1
127,Developer Controls Process,Work Flows Inward,Make the Developer the focal point of process ...,An organization must seek a structure that bes...,1
128,Developer Controls Process,Work Queue,Make the Developer the focal point of process ...,"It is difficult to do linear, monochronic sche...",1
281,Informal Labor Plan,Take No Small Slips,Let individuals devise their own short-term pl...,It's difficult to know how long a project shou...,1
352,Named Stable Bases,Private World,Stabilize system interfaces â€” the architectu...,How can we balance the need for developers to ...,1
368,Private World,Developer Controls Process,Provide a mechanism where developers can maint...,"A development culture, like any culture, can b...",1
411,Programming Episode,Take No Small Slips,Develop a program in discrete episodes. Select...,It's difficult to know how long a project shou...,1
481,Size The Schedule,Named Stable Bases,Reward developers for negotiating a schedule t...,It is important to integrate software frequent...,1


## Použitie Neurónovej siete

Zo získaných dát som sa pokúsil spraviť demo neúronovej siete na klasifikáciu dvojíc textov. Teda podobnosť medzi konečným kontextom (resulting_context) vzoru X a začiatočným kontextom (starting_context) vzoru Y. Kde ak tieto kontexy sú podobné, je možné povedať že vzor Y môže byť použitý po vzore X. Zároveň by sieť pracovala s informáciou o tom či už boli v sekvencií čo som robil v prechádzajúcom úseku kódu.

Avšak model, ktorý by dokázal pozerať na sémantiku textov, v mojom prípade to bol BERT respektíve model na ňom založený, dokázal zobrať iba texty do veľkosti 509, čo je príliš málo na moje textové opisy. Existujú riešenia založené na modeli BERT, ktoré zvládnu aj dlhšie texty ako BigBird alebo LongFormer, tie sa mi však nepodarilo spojazdniť zatiaľ. 

In [176]:
max(df_final["starting_context"].apply(len))

1506

In [216]:
val_df["starting_context"].apply(len)

147     324
148    1506
149     232
151     861
152    1346
153     663
154    1003
155    1075
156     492
157     709
158    1281
159     644
160     476
161     324
162    1506
163     232
164     449
166    1346
167     663
168    1003
169    1075
170     492
171     709
172    1281
173     644
174     476
175     324
176    1506
177     232
178     449
179     861
181     663
182    1003
183    1075
184     492
185     709
186    1281
187     644
188     476
189     324
190    1506
191     232
192     449
193     861
194    1346
Name: starting_context, dtype: int64

## Iné riešenie (bez neurónovej siete)

Za použitia BERT modelu all-mpnet-base-v2, ktorý je natrénovaný a pripravený na použitie, viem vytvoriť vektorovú reprezentáciu textu, kde je zachovaná aj jeho sémantika. Vektory oboch kontextov viem porovnávať na základe metrík kosínusová podobnosť alebo euklidovksá vzdialenosť. Tieto mezriky podobností vektorov môžem použiť ako vstupy do ML modelu.

Rátam podobnosť medzi počiatočným a konečným kontextom, oboma začiatočnými kontextami a oboma konečnými kontextami.

In [11]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')


In [145]:
df_final = df_new
df_final

,pattern_name1,pattern_name2,resulting_context1,resulting_context2,starting_context1,starting_context2
0,Build Prototypes,Community Of Trust,Build an isolated prototype solution whose pur...,"Do things that explicitly demonstrate trust, s...",A project must test requirements and design de...,It is essential that the people in a team trus...
1,Build Prototypes,Completion Headroom,Build an isolated prototype solution whose pur...,Project work group completion dates from remai...,A project must test requirements and design de...,Every project must commit to delivery on a few...
2,Build Prototypes,Day Care,Build an isolated prototype solution whose pur...,"Put one expert in charge of all the novices, l...",A project must test requirements and design de...,Your experts are spending all their time mento...
3,Build Prototypes,Developer Controls Process,Build an isolated prototype solution whose pur...,Make the Developer the focal point of process ...,A project must test requirements and design de...,"A development culture, like any culture, can b..."
4,Build Prototypes,Development Episode,Build an isolated prototype solution whose pur...,Approach all development as a group activity a...,A project must test requirements and design de...,It's important to build on the collective stre...
...,...,...,...,...,...,...
1426,urrogate Customer,Unity Of Purpose,Create a SurrogateCustomer role in the project...,The leader of the project must instill a commo...,It is important to exchange ideas and clarify ...,Many projects have rocky beginnings as people ...
1427,urrogate Customer,Wise Fool,Create a SurrogateCustomer role in the project...,"Nurture the role of the wise fool, who can rai...",It is important to exchange ideas and clarify ...,Interpersonal dynamics often discourage good i...
1428,Team Pride,Unity Of Purpose,"""We're the best."" Instill a sense of elitism i...",The leader of the project must instill a commo...,People are most successful when they feel good...,Many projects have rocky beginnings as people ...
1429,Team Pride,Wise Fool,"""We're the best."" Instill a sense of elitism i...","Nurture the role of the wise fool, who can rai...",People are most successful when they feel good...,Interpersonal dynamics often discourage good i...


In [146]:
#vytváranie vektorových reprezentácií kontextov
df_final['starting_embedding1'] = df_final['starting_context1'].apply(model.encode)
df_final['resulting_embedding1'] = df_final['resulting_context1'].apply(model.encode)
df_final['starting_embedding2'] = df_final['starting_context2'].apply(model.encode)
df_final['resulting_embedding2'] = df_final['resulting_context2'].apply(model.encode)

In [62]:
df_final

,pattern_name1,pattern_name2,resulting_context1,resulting_context2,starting_context1,starting_context2,starting_embedding1,resulting_embedding1,starting_embedding2,resulting_embedding2
0,Build Prototypes,Community Of Trust,Build an isolated prototype solution whose pur...,"Do things that explicitly demonstrate trust, s...",A project must test requirements and design de...,It is essential that the people in a team trus...,"[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.033992566, 0.0021839319, -0.0064396067, 0.0...","[0.013892247, 0.0677837, -0.011958739, 0.01426..."
1,Build Prototypes,Completion Headroom,Build an isolated prototype solution whose pur...,Project work group completion dates from remai...,A project must test requirements and design de...,Every project must commit to delivery on a few...,"[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[-0.06432862, 0.046334974, -0.015850563, -0.08...","[-0.02881442, 0.105386145, -0.0058587473, -0.0..."
2,Build Prototypes,Day Care,Build an isolated prototype solution whose pur...,"Put one expert in charge of all the novices, l...",A project must test requirements and design de...,Your experts are spending all their time mento...,"[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.069971845, 0.056567065, -0.00016744515, -0....","[0.071074046, -0.003414135, -0.03895503, 0.012..."
3,Build Prototypes,Developer Controls Process,Build an isolated prototype solution whose pur...,Make the Developer the focal point of process ...,A project must test requirements and design de...,"A development culture, like any culture, can b...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.04957857, 0.05668401, -0.046187162, -0.0080...","[0.04166916, -0.056860633, -0.026997551, -0.03..."
4,Build Prototypes,Development Episode,Build an isolated prototype solution whose pur...,Approach all development as a group activity a...,A project must test requirements and design de...,It's important to build on the collective stre...,"[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.0035809937, 0.0012670304, -0.019009784, 0.0...","[0.048495773, 0.012811256, -0.018782204, -0.00..."
...,...,...,...,...,...,...,...,...,...,...
320,Team Per Task,Work Queue,One approach is to split the team. Sort the ac...,Produce a schedule that is simply a prioritize...,Large distractions (usually called crises) mus...,"It is difficult to do linear, monochronic sche...","[0.0011973408, 0.04903567, 0.011451123, -0.010...","[0.021445733, -0.005875324, -0.059443455, 0.04...","[0.023785634, 0.08178888, -0.02493737, -0.0035...","[0.023706136, 0.051402166, -0.029743375, -0.00..."
321,Team Per Task,Work Split,One approach is to split the team. Sort the ac...,Divide a task into an urgent and deferred comp...,Large distractions (usually called crises) mus...,A work group has an obligation to make its eff...,"[0.0011973408, 0.04903567, 0.011451123, -0.010...","[0.021445733, -0.005875324, -0.059443455, 0.04...","[0.015930163, 0.03433592, -0.0064756554, -0.05...","[0.0035704507, 0.05868317, -0.023359573, -0.02..."
322,Work Flows Inward,Work Queue,Work should flow in to developer from stakehol...,Produce a schedule that is simply a prioritize...,An organization must seek a structure that bes...,"It is difficult to do linear, monochronic sche...","[0.04887739, 0.0011259597, -0.029870065, -0.04...","[0.03404403, 0.020410828, -0.044784002, 0.0222...","[0.023785634, 0.08178888, -0.02493737, -0.0035...","[0.023706136, 0.051402166, -0.029743375, -0.00..."
323,Work Flows Inward,Work Split,Work should flow in to developer from stakehol...,Divide a task into an urgent and deferred comp...,An organization must seek a structure that bes...,A work group has an obligation to

In [149]:
#počítanie kosínusovej podobnosti

cos_sim1 = []
for pairs in df_final[['resulting_embedding1','starting_embedding2']].values:
    cos_sim1.append(cosine_similarity([pairs[0],pairs[1]]))
cos_values1 = []
for values in cos_sim1:
    cos_values1.append(values[0][1])
    
cos_sim2 = []
for pairs in df_final[['starting_embedding1','starting_embedding2']].values:
    cos_sim2.append(cosine_similarity([pairs[0],pairs[1]]))
cos_values2 = []
for values in cos_sim2:
    cos_values2.append(values[0][1])
    
cos_sim3 = []
for pairs in df_final[['resulting_embedding1','resulting_embedding2']].values:
    cos_sim3.append(cosine_similarity([pairs[0],pairs[1]]))
cos_values3 = []
for values in cos_sim3:
    cos_values3.append(values[0][1])
    
cos_sim4 = []
for pairs in df_final[['starting_embedding1','resulting_embedding2']].values:
    cos_sim4.append(cosine_similarity([pairs[0],pairs[1]]))
cos_values4 = []
for values in cos_sim4:
    cos_values4.append(values[0][1])

In [150]:
df_final['cos_similarity_result_start_1']=cos_values1
df_final['cos_similarity_start_start']=cos_values2
df_final['cos_similarity_result_result']=cos_values3
df_final['cos_similarity_result_start_2']=cos_values4

In [409]:
df_final['euc_distance']=euc_values

C:\Users\olive\AppData\Local\Temp\ipykernel_4724\2520422237.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['euc_distance']=euc_values


Výsledná tabuľka s metrikami

In [151]:
df_final

,pattern_name1,pattern_name2,resulting_context1,resulting_context2,starting_context1,starting_context2,starting_embedding1,resulting_embedding1,starting_embedding2,resulting_embedding2,cos_similarity_result_start_1,cos_similarity_start_start,cos_similarity_result_result,cos_similarity_result_start_2
0,Build Prototypes,Community Of Trust,Build an isolated prototype solution whose pur...,"Do things that explicitly demonstrate trust, s...",A project must test requirements and design de...,It is essential that the people in a team trus...,"[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.033992566, 0.0021839319, -0.0064396067, 0.0...","[0.013892247, 0.0677837, -0.011958739, 0.01426...",0.252579,0.399197,0.249860,0.350796
1,Build Prototypes,Completion Headroom,Build an isolated prototype solution whose pur...,Project work group completion dates from remai...,A project must test requirements and design de...,Every project must commit to delivery on a few...,"[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[-0.06432862, 0.046334974, -0.015850563, -0.08...","[-0.02881442, 0.105386145, -0.0058587473, -0.0...",0.147638,0.287624,0.218520,0.267191
2,Build Prototypes,Day Care,Build an isolated prototype solution whose pur...,"Put one expert in charge of all the novices, l...",A project must test requirements and design de...,Your experts are spending all their time mento...,"[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.069971845, 0.056567065, -0.00016744515, -0....","[0.071074046, -0.003414135, -0.03895503, 0.012...",0.224507,0.230168,0.334633,0.230830
3,Build Prototypes,Developer Controls Process,Build an isolated prototype solution whose pur...,Make the Developer the focal point of process ...,A project must test requirements and design de...,"A development culture, like any culture, can b...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.04957857, 0.05668401, -0.046187162, -0.0080...","[0.04166916, -0.056860633, -0.026997551, -0.03...",0.277095,0.514923,0.239971,0.340766
4,Build Prototypes,Development Episode,Build an isolated prototype solution whose pur...,Approach all development as a group activity a...,A project must test requirements and design de...,It's important to build on the collective stre...,"[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.0035809937, 0.0012670304, -0.019009784, 0.0...","[0.048495773, 0.012811256, -0.018782204, -0.00...",0.064959,0.175616,0.341376,0.349847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1426,urrogate Customer,Unity Of Purpose,Create a SurrogateCustomer role in the project...,The leader of the project must instill a commo...,It is important to exchange ideas and clarify ...,Many projects have rocky beginnings as people ...,"[0.07129714, 0.05089531, -0.022737782, 0.02519...","[0.039613444, 0.06593218, -0.027706152, 0.0065...","[0.03618246, 0.029766077, -0.025872428, -1.552...","[0.05665192, 0.033690587, 0.00075172447, 0.032...",0.396681,0.392890,0.450400,0.375113
1427,urrogate Customer,Wise Fool,Create a SurrogateCustomer role in the project...,"Nurture the role of the wise fool, who can rai...",It is important to exchange ideas and clarify ...,Interpersonal dynamics often discourage good i...,"[0.07129714, 0.05089531, -0.022737782, 0.02519...","[0.039613444, 0.06593218, -0.027706152, 0.0065...","[0.03659603, 0.013783946, 0.027074216, 0.01551...","[0.04201864, 0.072648905, 0.027401034, 0.02183...",0.203926,0.287803,0.309779,0.281045
1428,Team Pride,Unity Of Purpose,"""We're the best."" Instill a sense of elitism i...",The leader of the project must instill a commo...,People are most successful when they feel good...,Many projects have rocky beginnings as people ...,"[0.00018179686, 0.0458

## Podobnosti konečného kontextu vzoru X a začiatočného vzoru Y
Najväčšiu podobnosť mali vzory CompletionHeadRoom a WorkSplit čo keď sa pozrieme do ích textových opisov tak je vidieť, že tam naozaj existuje nejaký vzťah.

Po pridaní ďalšieho jazyka vzorov vznikol ďalšia silná podobnosť medzi EngageCustomers a SurrogateCustomer.

In [152]:
df_final.sort_values(by=['cos_similarity_result_start_1'],ascending=False).head(10)

,pattern_name1,pattern_name2,resulting_context1,resulting_context2,starting_context1,starting_context2,starting_embedding1,resulting_embedding1,starting_embedding2,resulting_embedding2,cos_similarity_result_start_1,cos_similarity_start_start,cos_similarity_result_result,cos_similarity_result_start_2
1196,EngageCustomers,urrogate Customer,Closely couple the Customer role to the Develo...,Create a SurrogateCustomer role in the project...,It's important that the development organizati...,It is important to exchange ideas and clarify ...,"[0.08553004, 0.04444293, -0.029072814, -0.0267...","[0.087259285, 0.014686615, -0.046560932, 0.028...","[0.07129714, 0.05089531, -0.022737782, 0.02519...","[0.039613444, 0.06593218, -0.027706152, 0.0065...",0.754471,0.771849,0.695268,0.652926
126,Completion Headroom,Work Split,Project work group completion dates from remai...,Divide a task into an urgent and deferred comp...,Every project must commit to delivery on a few...,A work group has an obligation to make its eff...,"[-0.06432862, 0.046334974, -0.015850563, -0.08...","[-0.02881442, 0.105386145, -0.0058587473, -0.0...","[0.015930163, 0.03433592, -0.0064756554, -0.05...","[0.0035704507, 0.05868317, -0.023359573, -0.02...",0.727226,0.311419,0.454712,0.391157
1203,Engage Quality Assurance,Group Validation,Make QA a central role. Couple it tightly to d...,"Even before engaging QA, the development team ...",Customer engagement is a key element of qualit...,Product quality is crucial to the success of t...,"[0.050087288, 0.05367348, -0.05450759, 0.00260...","[0.032119405, 0.05401316, -0.026115092, 0.0091...","[0.043183528, 0.023295484, -0.037826177, 0.007...","[0.04413286, 0.012007737, -0.014635134, -0.007...",0.705157,0.618322,0.646996,0.632082
1146,Diverse Groups,Self-Selecting Team,Consider temperaments and diverse experience b...,Create enthusiastic teams by letting people se...,Homogeneous teams that comprise too many of th...,The worst team dynamics can be found in appoin...,"[0.050589025, 0.024854558, -0.020808082, -0.01...","[0.024144419, 0.022501651, -0.011508377, 0.014...","[0.033132505, 0.029942734, -0.041641798, 0.013...","[0.022171535, 0.044167697, -0.03011777, 0.0415...",0.678630,0.636739,0.652056,0.435856
301,Development Episode,Unity Of Purpose,Approach all development as a group activity a...,The leader of the project must instill a commo...,It's important to build on the collective stre...,Many projects have rocky beginnings as people ...,"[0.0035809937, 0.0012670304, -0.019009784, 0.0...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[0.03618246, 0.029766077, -0.025872428, -1.552...","[0.05665192, 0.033690587, 0.00075172447, 0.032...",0.666686,0.383577,0.601373,0.413898
918,Team Per Task,Holistic Diversity,One approach is to split the team. Sort the ac...,For each function or set of functions to be de...,Large distractions (usually called crises) mus...,"Development of a subsystem needs many skills, ...","[0.0011973408, 0.04903567, 0.011451123, -0.010...","[0.021445733, -0.005875324, -0.059443455, 0.04...","[0.009766198, 0.031992897, -0.040107384, 0.000...","[0.060784, 0.031808477, -0.017394414, -0.00461...",0.652153,0.363399,0.528141,0.254887
601,Mercenary Analyst,Scenarios Define Problem,"Hire a technical writer, proficient in the nec...",Capture system functional requirements as use ...,Technical documentation is the dirty work ever...,Design documents are often ineffective as vehi...,"[0.020133274, -0.014507973, -0.016232308, 0.00...","[0.008096245, 0.038123347, -0.0067579434, -0.0...","[0.044110455, 0.031120244, -0.031288765, 0.023...","[0.05868088, 0.027389029, 0.008235332, 0.00170...",0.622375,0.633717,0.326217,0.348644
1190,EngageCustomers,Scenarios Define Problem,Closely couple the Customer role to the Develo...,Capture system functional requirements as use ...,It's important that the development organizati...,Design documents are often ineffective as vehi...,"[0.08553004, 0.04444293, -0.029072814, -0.0267...","

Naopak najmenej podobné kontexty mali MercenaryAnalyst a NamedStableBases.

Po pridaní ďalšieho jazyka vzorov vznikol ďalšia veľmi slabá podobnosť medzi InterruptsUnjamBlocking a SoloVirtuoso.

In [153]:
df_final.sort_values(by=['cos_similarity_result_start_1']).head(10)

,pattern_name1,pattern_name2,resulting_context1,resulting_context2,starting_context1,starting_context2,starting_embedding1,resulting_embedding1,starting_embedding2,resulting_embedding2,cos_similarity_result_start_1,cos_similarity_start_start,cos_similarity_result_result,cos_similarity_result_start_2
564,Interrupts Unjam Blocking,Solo Virtuoso,If a role is about to block on a critical reso...,Do the entire design and implementation with o...,A comprehensive scheduling plan is difficult i...,When a smaller software project (less than twe...,"[-0.0021273368, 0.07085707, -0.023154281, -0.0...","[0.009520725, -0.01611056, -0.0016354743, -0.0...","[0.011190042, 0.03326897, -0.012879533, -0.068...","[0.042240884, 0.066174544, -0.028253678, 0.019...",-0.010899,0.353839,0.199666,0.273364
885,Take No Small Slips,Group Validation,Prefer a single large slip to several small sl...,"Even before engaging QA, the development team ...",It's difficult to know how long a project shou...,Product quality is crucial to the success of t...,"[0.015812408, 0.10102768, -0.009773148, -0.043...","[-0.018102376, -0.0074234153, 0.008787064, -0....","[0.043183528, 0.023295484, -0.037826177, 0.007...","[0.04413286, 0.012007737, -0.014635134, -0.007...",-0.009680,0.212549,0.019112,0.294899
772,Sacrifice One Person,Application Design Is Bounded By Test Design,Assign just one person to it until it gets han...,Use case-driven test design starts when the cu...,"Small distractions can add up, and sap the str...",When do you design and implement test plans an...,"[0.010107814, -0.04778266, -0.017414702, -0.01...","[0.00020712917, -0.003472787, -0.018456511, 0....","[-0.022668341, -0.03599587, -0.019662376, -0.0...","[0.022802858, -0.008028176, -0.03563411, -0.01...",-0.006683,0.012333,0.016948,0.025686
1052,Application Design Is Bounded By Test Design,Wise Fool,Use case-driven test design starts when the cu...,"Nurture the role of the wise fool, who can rai...",When do you design and implement test plans an...,Interpersonal dynamics often discourage good i...,"[-0.022668341, -0.03599587, -0.019662376, -0.0...","[0.022802858, -0.008028176, -0.03563411, -0.01...","[0.03659603, 0.013783946, 0.027074216, 0.01551...","[0.04201864, 0.072648905, 0.027401034, 0.02183...",-0.005077,0.132434,0.100973,0.105599
1059,Apprenticeship,Failed Project Wake,Turn new hires into experts (see DomainExperti...,Hold a wake for the failed project. This shoul...,A project must balance its need for growth wit...,"Canceling a project, even for the best externa...","[0.03503278, 0.026869062, -0.012360971, -0.025...","[0.0432205, 0.04235784, 0.0030211227, -0.03399...","[-0.0326711, 0.05866426, 0.010470345, -0.00173...","[-0.038515694, 0.13793832, 0.009984291, 0.0018...",0.000786,0.303404,0.072112,0.285142
882,Take No Small Slips,Failed Project Wake,Prefer a single large slip to several small sl...,Hold a wake for the failed project. This shoul...,It's difficult to know how long a project shou...,"Canceling a project, even for the best externa...","[0.015812408, 0.10102768, -0.009773148, -0.043...","[-0.018102376, -0.0074234153, 0.008787064, -0....","[-0.0326711, 0.05866426, 0.010470345, -0.00173...","[-0.038515694, 0.13793832, 0.009984291, 0.0018...",0.001967,0.379410,0.046117,0.265437
1032,Application Design Is Bounded By Test Design,Failed Project Wake,Use case-driven test design starts when the cu...,Hold a wake for the failed project. This shoul...,When do you design and implement test plans an...,"Canceling a project, even for the best externa...","[-0.022668341, -0.03599587, -0.019662376, -0.0...","[0.022802858, -0.008028176, -0.03563411, -0.01...","[-0.0326711, 0.05866426, 0.010470345, -0.00173...","[-0.038515694, 0.13793832, 0.009984291, 0.0018...",0.005775,0.006782,0.067180,0.100044
570,Mercenary Analyst,Named Stable Bases,"Hire a technical writer, proficient in the nec...",Stabilize system interfaces â€” the architectu...,Technical documentation is the dirty work ever...,It is importa

## Podobnosti konečného kontextu vzoru Y a začiatočného vzoru X
Najväčšiu podobnosť mali vzory DeveloperControllsProcess a WorkFlowsInward.

In [194]:
df_final.sort_values(by=['cos_similarity_result_start_2'],ascending=False).head(10)

,pattern_name1,pattern_name2,resulting_context1,resulting_context2,starting_context1,starting_context2,starting_embedding1,resulting_embedding1,starting_embedding2,resulting_embedding2,cos_similarity_result_start_1,cos_similarity_start_start,cos_similarity_result_result,cos_similarity_result_start_2,result_start_1,result_start_2,start_start,result_result
223,Developer Controls Process,Work Flows Inward,Make the Developer the focal point of process ...,Work should flow in to developer from stakehol...,"A development culture, like any culture, can b...",An organization must seek a structure that bes...,"[0.04957857, 0.05668401, -0.046187162, -0.0080...","[0.04166916, -0.056860633, -0.026997551, -0.03...","[0.04887739, 0.0011259597, -0.029870065, -0.04...","[0.03404403, 0.020410828, -0.044784002, 0.0222...",0.509857,0.686973,0.470207,0.655913,0,1,1,0
1196,EngageCustomers,urrogate Customer,Closely couple the Customer role to the Develo...,Create a SurrogateCustomer role in the project...,It's important that the development organizati...,It is important to exchange ideas and clarify ...,"[0.08553004, 0.04444293, -0.029072814, -0.0267...","[0.087259285, 0.014686615, -0.046560932, 0.028...","[0.07129714, 0.05089531, -0.022737782, 0.02519...","[0.039613444, 0.06593218, -0.027706152, 0.0065...",0.754471,0.771849,0.695268,0.652926,1,1,1,1
1056,Apprenticeship,Domain Expertise In Roles,Turn new hires into experts (see DomainExperti...,"Hire domain experts with proven track records,...",A project must balance its need for growth wit...,Matching staff with roles is one of the hardes...,"[0.03503278, 0.026869062, -0.012360971, -0.025...","[0.0432205, 0.04235784, 0.0030211227, -0.03399...","[0.033357415, 0.02620629, -0.018051252, -0.044...","[0.06965132, 0.02252271, -0.03190883, 0.007860...",0.361728,0.605948,0.530066,0.652040,0,1,1,0
1203,Engage Quality Assurance,Group Validation,Make QA a central role. Couple it tightly to d...,"Even before engaging QA, the development team ...",Customer engagement is a key element of qualit...,Product quality is crucial to the success of t...,"[0.050087288, 0.05367348, -0.05450759, 0.00260...","[0.032119405, 0.05401316, -0.026115092, 0.0091...","[0.043183528, 0.023295484, -0.037826177, 0.007...","[0.04413286, 0.012007737, -0.014635134, -0.007...",0.705157,0.618322,0.646996,0.632082,1,1,1,1
278,Development Episode,Diverse Groups,Approach all development as a group activity a...,Consider temperaments and diverse experience b...,It's important to build on the collective stre...,Homogeneous teams that comprise too many of th...,"[0.0035809937, 0.0012670304, -0.019009784, 0.0...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[0.050589025, 0.024854558, -0.020808082, -0.01...","[0.024144419, 0.022501651, -0.011508377, 0.014...",0.577250,0.485816,0.552055,0.620068,0,1,0,0
1428,Team Pride,Unity Of Purpose,"""We're the best."" Instill a sense of elitism i...",The leader of the project must instill a commo...,People are most successful when they feel good...,Many projects have rocky beginnings as people ...,"[0.00018179686, 0.04582352, 0.001021349, 0.009...","[0.024695326, 0.011348519, 0.022361487, 0.0477...","[0.03618246, 0.029766077, -0.025872428, -1.552...","[0.05665192, 0.033690587, 0.00075172447, 0.032...",0.393009,0.576701,0.658926,0.611281,0,1,0,1
532,Interrupts Unjam Blocking,Recommitment Meeting,If a role is about to block on a critical reso...,Assemble a meeting of interested management an...,A comprehensive scheduling plan is difficult i...,If a Product Initiative is in jeopardy because...,"[-0.0021273368, 0.07085707, -0.023154281, -0.0...","[0.009520725, -0.01611056, -0.0016354743, -0.0...","[0.010786134, 0.10886827, 0.003984304, -0.0595...","[-0.011295064, 0.060159247, -0.019415945, -0.0...",0.371767,0.309565,0.261467,0.598939,0,0,0,0
43,Build Prototypes,Scenarios Define Problem,Build an isolated prototype solution whose pur...,Capture system functional requirements as use ...,A project must test 

Naopak najmenej podobné kontexty mali ApplicationDesignIsBoundedByTestDesign a PatronRole.

In [195]:
df_final.sort_values(by=['cos_similarity_result_start_2']).head(10)

,pattern_name1,pattern_name2,resulting_context1,resulting_context2,starting_context1,starting_context2,starting_embedding1,resulting_embedding1,starting_embedding2,resulting_embedding2,cos_similarity_result_start_1,cos_similarity_start_start,cos_similarity_result_result,cos_similarity_result_start_2,result_start_1,result_start_2,start_start,result_result
1040,Application Design Is Bounded By Test Design,Patron Role,Use case-driven test design starts when the cu...,"Give the project access to a visible, high-lev...",When do you design and implement test plans an...,It is important to give a project continuity. ...,"[-0.022668341, -0.03599587, -0.019662376, -0.0...","[0.022802858, -0.008028176, -0.03563411, -0.01...","[0.0020334988, 0.09311888, -0.009235093, -0.02...","[0.021286698, 0.051788032, 0.0002978026, 0.053...",0.081237,0.164928,0.066896,-0.003953,0,0,0,0
1236,Failed Project Wake,Subsystem By Skill,Hold a wake for the failed project. This shoul...,Separate subsystems by staff skills and skill ...,"Canceling a project, even for the best externa...",Birds of a feather flock together. By ConwaysL...,"[-0.0326711, 0.05866426, 0.010470345, -0.00173...","[-0.038515694, 0.13793832, 0.009984291, 0.0018...","[0.0250982, 0.033371083, -0.0030294019, -0.012...","[0.019256314, -0.011144504, -0.041393787, 0.00...",0.098575,0.102388,0.064086,0.013852,0,0,0,0
993,Work Queue,Team Pride,Produce a schedule that is simply a prioritize...,"""We're the best."" Instill a sense of elitism i...","It is difficult to do linear, monochronic sche...",People are most successful when they feel good...,"[0.023785634, 0.08178888, -0.02493737, -0.0035...","[0.023706136, 0.051402166, -0.029743375, -0.00...","[0.00018179686, 0.04582352, 0.001021349, 0.009...","[0.024695326, 0.011348519, 0.022361487, 0.0477...",0.217191,0.194132,0.109805,0.023218,0,0,0,0
772,Sacrifice One Person,Application Design Is Bounded By Test Design,Assign just one person to it until it gets han...,Use case-driven test design starts when the cu...,"Small distractions can add up, and sap the str...",When do you design and implement test plans an...,"[0.010107814, -0.04778266, -0.017414702, -0.01...","[0.00020712917, -0.003472787, -0.018456511, 0....","[-0.022668341, -0.03599587, -0.019662376, -0.0...","[0.022802858, -0.008028176, -0.03563411, -0.01...",-0.006683,0.012333,0.016948,0.025686,0,0,0,0
16,Build Prototypes,Sacrifice One Person,Build an isolated prototype solution whose pur...,Assign just one person to it until it gets han...,A project must test requirements and design de...,"Small distractions can add up, and sap the str...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.010107814, -0.04778266, -0.017414702, -0.01...","[0.00020712917, -0.003472787, -0.018456511, 0....",0.052221,0.141461,0.111258,0.025786,0,0,0,0
1034,Application Design Is Bounded By Test Design,Gate Keeper,Use case-driven test design starts when the cu...,"One project member, a PublicCharacter with an ...",When do you design and implement test plans an...,A project must develop good interfaces with th...,"[-0.022668341, -0.03599587, -0.019662376, -0.0...","[0.022802858, -0.008028176, -0.03563411, -0.01...","[0.0013977767, 0.061598483, -0.021921508, 0.00...","[0.036360897, 0.07367269, -0.0060292576, 0.013...",0.163675,0.228657,0.113079,0.026636,0,0,0,0
995,Work Queue,Wise Fool,Produce a schedule that is simply a prioritize...,"Nurture the role of the wise fool, who can rai...","It is difficult to do linear, monochronic sche...",Interpersonal dynamics often discourage good i...,"[0.023785634, 0.08178888, -0.02493737, -0.0035...","[0.023706136, 0.051402166, -0.029743375, -0.00...","[0.03659603, 0.013783946, 0.027074216, 0.01551...","[0.04201864, 0.072648905, 0.027401034, 0.02183...",0.138955,0.084831,0.151450,0.027779,0,0,0,0
177,Day Care,Application Design Is Bounded By Test Design,"Put one expert in charge of all the novices, l...",Use case-driven t

## Podobnosti začiatočného kontextu vzoru X a začiatočného vzoru Y


In [191]:
df_final.sort_values(by=['cos_similarity_start_start'],ascending=False).head(10)

,pattern_name1,pattern_name2,resulting_context1,resulting_context2,starting_context1,starting_context2,starting_embedding1,resulting_embedding1,starting_embedding2,resulting_embedding2,cos_similarity_result_start_1,cos_similarity_start_start,cos_similarity_result_result,cos_similarity_result_start_2,result_start_1,result_start_2,start_start,result_result
1404,Size The Organization,Solo Virtuoso,"By default, choose about ten people to establi...",Do the entire design and implementation with o...,Large software projects (greater than twenty-f...,When a smaller software project (less than twe...,"[0.049533434, 0.028160773, -0.027851027, -0.05...","[0.037290186, -0.0263837, -0.016910883, -0.052...","[0.011190042, 0.03326897, -0.012879533, -0.068...","[0.042240884, 0.066174544, -0.028253678, 0.019...",0.468928,0.796201,0.447995,0.422041,0,0,1,0
1196,EngageCustomers,urrogate Customer,Closely couple the Customer role to the Develo...,Create a SurrogateCustomer role in the project...,It's important that the development organizati...,It is important to exchange ideas and clarify ...,"[0.08553004, 0.04444293, -0.029072814, -0.0267...","[0.087259285, 0.014686615, -0.046560932, 0.028...","[0.07129714, 0.05089531, -0.022737782, 0.02519...","[0.039613444, 0.06593218, -0.027706152, 0.0065...",0.754471,0.771849,0.695268,0.652926,1,1,1,1
802,Size The Schedule,Take No Small Slips,Reward developers for negotiating a schedule t...,Prefer a single large slip to several small sl...,Both overly ambitious schedules and overly gen...,It's difficult to know how long a project shou...,"[0.020185143, 0.10455826, -0.008474356, -0.041...","[0.020484738, 0.051352836, -0.035016783, -0.00...","[0.015812408, 0.10102768, -0.009773148, -0.043...","[-0.018102376, -0.0074234153, 0.008787064, -0....",0.555571,0.770337,0.279796,0.394989,0,0,1,0
1068,Apprenticeship,Phasing It In,Turn new hires into experts (see DomainExperti...,Phase the hiring program. Start by hiring peop...,A project must balance its need for growth wit...,Growing projects must figure out how to grow l...,"[0.03503278, 0.026869062, -0.012360971, -0.025...","[0.0432205, 0.04235784, 0.0030211227, -0.03399...","[0.018057844, 0.00972775, 0.0013048564, -0.058...","[0.04751829, 0.05801926, -0.01015259, -0.04318...",0.313999,0.770281,0.488309,0.574087,0,0,1,0
1241,Fire Walls,Gate Keeper,"Create a ManagerRole, who shields other develo...","One project member, a PublicCharacter with an ...",It's important to placate stakeholders who fee...,A project must develop good interfaces with th...,"[0.012233101, 0.068203, -0.033718087, 0.039389...","[0.028908951, 0.05352523, 0.0076098167, 0.0191...","[0.0013977767, 0.061598483, -0.021921508, 0.00...","[0.036360897, 0.07367269, -0.0060292576, 0.013...",0.433470,0.767022,0.448159,0.452868,0,0,1,0
1190,EngageCustomers,Scenarios Define Problem,Closely couple the Customer role to the Develo...,Capture system functional requirements as use ...,It's important that the development organizati...,Design documents are often ineffective as vehi...,"[0.08553004, 0.04444293, -0.029072814, -0.0267...","[0.087259285, 0.014686615, -0.046560932, 0.028...","[0.044110455, 0.031120244, -0.031288765, 0.023...","[0.05868088, 0.027389029, 0.008235332, 0.00170...",0.611008,0.727601,0.452648,0.451167,1,0,1,0
539,Interrupts Unjam Blocking,Work Queue,If a role is about to block on a critical reso...,Produce a schedule that is simply a prioritize...,A comprehensive scheduling plan is difficult i...,"It is difficult to do linear, monochronic sche...","[-0.0021273368, 0.07085707, -0.023154281, -0.0...","[0.009520725, -0.01611056, -0.0016354743, -0.0...","[0.023785634, 0.08178888, -0.02493737, -0.0035...","[0.023706136, 0.051402166, -0.029743375, -0.00...",0.151227,0.699147,0.239666,0.522241,0,0,1,0
223,Developer Controls Process,Work Flows Inward,Make the Developer the focal point of process ...,Work should flow in to developer from stakehol...,"A development culture, like any culture, can b..."

In [32]:
df_final.sort_values(by=['cos_similarity_start_start']).head(10)

,pattern_name1,pattern_name2,resulting_context,resulting_context1,resulting_context2,starting_context1,starting_context2,starting_embedding1,resulting_embedding1,resulting_embedding2,starting_embedding2,cos_similarity_result_start,cos_similarity_start_start,cos_similarity_result_result
391,Programming Episode,Community Of Trust,Develop a program in discrete episodes. Select...,Develop a program in discrete episodes. Select...,"Do things that explicitly demonstrate trust, s...",Programming is the act of deciding now what wi...,It is essential that the people in a team trus...,"[0.034314055, 0.04872385, -0.014165695, -0.047...","[0.013992956, 0.089137785, -0.031594567, -0.04...","[0.013892247, 0.0677837, -0.011958739, 0.01426...","[0.033992566, 0.0021839319, -0.0064396067, 0.0...",0.118411,0.086439,0.222910
41,Community Of Trust,Programming Episode,"Do things that explicitly demonstrate trust, s...","Do things that explicitly demonstrate trust, s...",Develop a program in discrete episodes. Select...,It is essential that the people in a team trus...,Programming is the act of deciding now what wi...,"[0.033992566, 0.0021839319, -0.0064396067, 0.0...","[0.013892247, 0.0677837, -0.011958739, 0.01426...","[0.013992956, 0.089137785, -0.031594567, -0.04...","[0.034314055, 0.04872385, -0.014165695, -0.047...",0.190853,0.086439,0.222910
132,Development Episode,Completion Headroom,Approach all development as a group activity a...,Approach all development as a group activity a...,Project work group completion dates from remai...,It's important to build on the collective stre...,Every project must commit to delivery on a few...,"[0.0035809937, 0.0012670304, -0.019009784, 0.0...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[-0.02881442, 0.105386145, -0.0058587473, -0.0...","[-0.06432862, 0.046334974, -0.015850563, -0.08...",0.145468,0.090668,0.393857
57,Completion Headroom,Development Episode,Project work group completion dates from remai...,Project work group completion dates from remai...,Approach all development as a group activity a...,Every project must commit to delivery on a few...,It's important to build on the collective stre...,"[-0.06432862, 0.046334974, -0.015850563, -0.08...","[-0.02881442, 0.105386145, -0.0058587473, -0.0...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[0.0035809937, 0.0012670304, -0.019009784, 0.0...",0.237150,0.090668,0.393857
154,Development Episode,Work Queue,Approach all development as a group activity a...,Approach all development as a group activity a...,Produce a schedule that is simply a prioritize...,It's important to build on the collective stre...,"It is difficult to do linear, monochronic sche...","[0.0035809937, 0.0012670304, -0.019009784, 0.0...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[0.023706136, 0.051402166, -0.029743375, -0.00...","[0.023785634, 0.08178888, -0.02493737, -0.0035...",0.205864,0.094205,0.277859
629,Work Queue,Development Episode,Produce a schedule that is simply a prioritize...,Produce a schedule that is simply a prioritize...,Approach all development as a group activity a...,"It is difficult to do linear, monochronic sche...",It's important to build on the collective stre...,"[0.023785634, 0.08178888, -0.02493737, -0.0035...","[0.023706136, 0.051402166, -0.029743375, -0.00...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[0.0035809937, 0.0012670304, -0.019009784, 0.0...",0.161577,0.094205,0.277859
627,Work Queue,Day Care,Produce a schedule that is simply a prioritize...,Produce a schedule that is simply a prioritize...,"Put one expert in charge of all the novices, l...","It is difficult to do linear, monochronic sche...",Your experts are spending all their time mento...,"[0.023785634, 0.08178888, -0.02493737, -0.0035...","[0.023706136, 0.051402166, -0.029743375, -0.00...","[0.071074046, -0.003414135, -0.03895503, 0.012...","[0.069971845, 0.056567065, -0.00016744515, -0....",0.180253,0.118957,0.261723
102,Day Care,Work Queue,"Put one exper

## Podobnosti konečného kontextu vzoru X a konečného vzoru Y


In [33]:
df_final.sort_values(by=['cos_similarity_result_result'],ascending=False).head(10)

,pattern_name1,pattern_name2,resulting_context,resulting_context1,resulting_context2,starting_context1,starting_context2,starting_embedding1,resulting_embedding1,resulting_embedding2,starting_embedding2,cos_similarity_result_start,cos_similarity_start_start,cos_similarity_result_result
265,Informal Labor Plan,Development Episode,Let individuals devise their own short-term pl...,Let individuals devise their own short-term pl...,Approach all development as a group activity a...,A schedule of developer work tasks can both as...,It's important to build on the collective stre...,"[0.017129539, 0.06161713, -0.04943146, -0.0459...","[-0.018673072, 0.08835107, -0.023630276, -0.00...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[0.0035809937, 0.0012670304, -0.019009784, 0.0...",0.383551,0.272391,0.689016
140,Development Episode,Informal Labor Plan,Approach all development as a group activity a...,Approach all development as a group activity a...,Let individuals devise their own short-term pl...,It's important to build on the collective stre...,A schedule of developer work tasks can both as...,"[0.0035809937, 0.0012670304, -0.019009784, 0.0...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[-0.018673072, 0.08835107, -0.023630276, -0.00...","[0.017129539, 0.06161713, -0.04943146, -0.0459...",0.384728,0.272391,0.689016
137,Development Episode,Get On With It,Approach all development as a group activity a...,Approach all development as a group activity a...,As soon as you have confidence about some proj...,It's important to build on the collective stre...,You can't wait until you have every last requi...,"[0.0035809937, 0.0012670304, -0.019009784, 0.0...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[0.036525305, 0.0789404, -0.017670574, -0.0470...","[0.048748564, 0.03481439, -0.0028045839, -0.06...",0.463513,0.317814,0.664009
187,Get On With It,Development Episode,As soon as you have confidence about some proj...,As soon as you have confidence about some proj...,Approach all development as a group activity a...,You can't wait until you have every last requi...,It's important to build on the collective stre...,"[0.048748564, 0.03481439, -0.0028045839, -0.06...","[0.036525305, 0.0789404, -0.017670574, -0.0470...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[0.0035809937, 0.0012670304, -0.019009784, 0.0...",0.396346,0.317814,0.664009
434,Recommitment Meeting,Size The Schedule,Assemble a meeting of interested management an...,Assemble a meeting of interested management an...,Reward developers for negotiating a schedule t...,If a Product Initiative is in jeopardy because...,Both overly ambitious schedules and overly gen...,"[0.010786134, 0.10886827, 0.003984304, -0.0595...","[-0.011295064, 0.060159247, -0.019415945, -0.0...","[0.020484738, 0.051352836, -0.035016783, -0.00...","[0.020185143, 0.10455826, -0.008474356, -0.041...",0.484778,0.495947,0.604407
484,Size The Schedule,Recommitment Meeting,Reward developers for negotiating a schedule t...,Reward developers for negotiating a schedule t...,Assemble a meeting of interested management an...,Both overly ambitious schedules and overly gen...,If a Product Initiative is in jeopardy because...,"[0.020185143, 0.10455826, -0.008474356, -0.041...","[0.020484738, 0.051352836, -0.035016783, -0.00...","[-0.011295064, 0.060159247, -0.019415945, -0.0...","[0.010786134, 0.10886827, 0.003984304, -0.0595...",0.448217,0.495947,0.604407
247,Incremental Integration,Named Stable Bases,Provide a mechanism to allow developers to bui...,Provide a mechanism to allow developers to bui...,Stabilize system interfaces â€” the architectu...,"For iterative development to work well, it is ...",It is important to integrate software frequent...,"[0.013965584, 0.01856928, -0.025939543, -0.033...","[-0.017080508, 0.059721574, -0.05366238, -0.04...","[-0.028876815, 0.034207717, -0.014542314, -0.0...","[-0.008631551, 0.04980158, -0.008348653, -0.05...",0.352588,0.454711,0.588751
347,Named Stable 

In [34]:
df_final.sort_values(by=['cos_similarity_result_result']).head(10)

,pattern_name1,pattern_name2,resulting_context,resulting_context1,resulting_context2,starting_context1,starting_context2,starting_embedding1,resulting_embedding1,resulting_embedding2,starting_embedding2,cos_similarity_result_start,cos_similarity_start_start,cos_similarity_result_result
549,Take No Small Slips,Day Care,Prefer a single large slip to several small sl...,Prefer a single large slip to several small sl...,"Put one expert in charge of all the novices, l...",It's difficult to know how long a project shou...,Your experts are spending all their time mento...,"[0.015812408, 0.10102768, -0.009773148, -0.043...","[-0.018102376, -0.0074234153, 0.008787064, -0....","[0.071074046, -0.003414135, -0.03895503, 0.012...","[0.069971845, 0.056567065, -0.00016744515, -0....",0.049555,0.361423,-0.027780
99,Day Care,Take No Small Slips,"Put one expert in charge of all the novices, l...","Put one expert in charge of all the novices, l...",Prefer a single large slip to several small sl...,Your experts are spending all their time mento...,It's difficult to know how long a project shou...,"[0.069971845, 0.056567065, -0.00016744515, -0....","[0.071074046, -0.003414135, -0.03895503, 0.012...","[-0.018102376, -0.0074234153, 0.008787064, -0....","[0.015812408, 0.10102768, -0.009773148, -0.043...",0.179358,0.361423,-0.027780
566,Take No Small Slips,Surrogate Customer,Prefer a single large slip to several small sl...,Prefer a single large slip to several small sl...,Create a SurrogateCustomer role in the project...,It's difficult to know how long a project shou...,It is important to exchange ideas and clarify ...,"[0.015812408, 0.10102768, -0.009773148, -0.043...","[-0.018102376, -0.0074234153, 0.008787064, -0....","[0.039613444, 0.06593218, -0.027706152, 0.0065...","[0.07129714, 0.05089531, -0.022737782, 0.02519...",0.029477,0.412247,-0.010316
541,Surrogate Customer,Take No Small Slips,Create a SurrogateCustomer role in the project...,Create a SurrogateCustomer role in the project...,Prefer a single large slip to several small sl...,It is important to exchange ideas and clarify ...,It's difficult to know how long a project shou...,"[0.07129714, 0.05089531, -0.022737782, 0.02519...","[0.039613444, 0.06593218, -0.027706152, 0.0065...","[-0.018102376, -0.0074234153, 0.008787064, -0....","[0.015812408, 0.10102768, -0.009773148, -0.043...",0.266741,0.412247,-0.010316
125,Developer Controls Process,Take No Small Slips,Make the Developer the focal point of process ...,Make the Developer the focal point of process ...,Prefer a single large slip to several small sl...,"A development culture, like any culture, can b...",It's difficult to know how long a project shou...,"[0.04957857, 0.05668401, -0.046187162, -0.0080...","[0.04166916, -0.056860633, -0.026997551, -0.03...","[-0.018102376, -0.0074234153, 0.008787064, -0....","[0.015812408, 0.10102768, -0.009773148, -0.043...",0.255678,0.320126,0.039159
550,Take No Small Slips,Developer Controls Process,Prefer a single large slip to several small sl...,Prefer a single large slip to several small sl...,Make the Developer the focal point of process ...,It's difficult to know how long a project shou...,"A development culture, like any culture, can b...","[0.015812408, 0.10102768, -0.009773148, -0.043...","[-0.018102376, -0.0074234153, 0.008787064, -0....","[0.04166916, -0.056860633, -0.026997551, -0.03...","[0.04957857, 0.05668401, -0.046187162, -0.0080...",0.098928,0.320126,0.039159
21,Build Prototypes,Take No Small Slips,Build an isolated prototype solution whose pur...,Build an isolated prototype solution whose pur...,Prefer a single large slip to several small sl...,A project must test requirements and design de...,It's difficult to know how long a project shou...,"[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[-0.018102376, -0.0074234153, 0.008787064, -0....","[0.015812408, 0.10102768, -0.009773148, -0.043...",0.176531,0.395928,0.051182
546,Take No

## Pridávanie vzťahov

3 vzťahy:

1 - konečný kontext vzoru X a začiatočný vzoru Y sú podobné1

2 - začiatočný kontext vzoru X a začiatočný vzoru Y sú podobné

3 - konečný kontext vzoru X a konečný vzoru Y sú podobné


In [158]:
df_final['result_start_1'] = 0
df_final['result_start_2'] = 0
df_final['start_start'] = 0
df_final['result_result'] = 0
df_final

,pattern_name1,pattern_name2,resulting_context1,resulting_context2,starting_context1,starting_context2,starting_embedding1,resulting_embedding1,starting_embedding2,resulting_embedding2,cos_similarity_result_start_1,cos_similarity_start_start,cos_similarity_result_result,cos_similarity_result_start_2,result_start_1,result_start_2,start_start,result_result
0,Build Prototypes,Community Of Trust,Build an isolated prototype solution whose pur...,"Do things that explicitly demonstrate trust, s...",A project must test requirements and design de...,It is essential that the people in a team trus...,"[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.033992566, 0.0021839319, -0.0064396067, 0.0...","[0.013892247, 0.0677837, -0.011958739, 0.01426...",0.252579,0.399197,0.249860,0.350796,0,0,0,0
1,Build Prototypes,Completion Headroom,Build an isolated prototype solution whose pur...,Project work group completion dates from remai...,A project must test requirements and design de...,Every project must commit to delivery on a few...,"[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[-0.06432862, 0.046334974, -0.015850563, -0.08...","[-0.02881442, 0.105386145, -0.0058587473, -0.0...",0.147638,0.287624,0.218520,0.267191,0,0,0,0
2,Build Prototypes,Day Care,Build an isolated prototype solution whose pur...,"Put one expert in charge of all the novices, l...",A project must test requirements and design de...,Your experts are spending all their time mento...,"[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.069971845, 0.056567065, -0.00016744515, -0....","[0.071074046, -0.003414135, -0.03895503, 0.012...",0.224507,0.230168,0.334633,0.230830,0,0,0,0
3,Build Prototypes,Developer Controls Process,Build an isolated prototype solution whose pur...,Make the Developer the focal point of process ...,A project must test requirements and design de...,"A development culture, like any culture, can b...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.04957857, 0.05668401, -0.046187162, -0.0080...","[0.04166916, -0.056860633, -0.026997551, -0.03...",0.277095,0.514923,0.239971,0.340766,0,0,0,0
4,Build Prototypes,Development Episode,Build an isolated prototype solution whose pur...,Approach all development as a group activity a...,A project must test requirements and design de...,It's important to build on the collective stre...,"[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.0035809937, 0.0012670304, -0.019009784, 0.0...","[0.048495773, 0.012811256, -0.018782204, -0.00...",0.064959,0.175616,0.341376,0.349847,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1426,urrogate Customer,Unity Of Purpose,Create a SurrogateCustomer role in the project...,The leader of the project must instill a commo...,It is important to exchange ideas and clarify ...,Many projects have rocky beginnings as people ...,"[0.07129714, 0.05089531, -0.022737782, 0.02519...","[0.039613444, 0.06593218, -0.027706152, 0.0065...","[0.03618246, 0.029766077, -0.025872428, -1.552...","[0.05665192, 0.033690587, 0.00075172447, 0.032...",0.396681,0.392890,0.450400,0.375113,0,0,0,0
1427,urrogate Customer,Wise Fool,Create a SurrogateCustomer role in the project...,"Nurture the role of the wise fool, who can rai...",It is important to exchange ideas and clarify ...,Interpersonal dynamics often discourage good i...,"[0.07129714, 0.05089531, -0.022737782, 0.02519...","[0.039613444, 0.06593218, -0.027706152, 0.0065...","[0.03659603, 0.013783946, 0.027074216, 0.01551...","[0.04201864, 0.072648905, 0.027401034, 0.02183...",0.203926,0.287803,0.309779,0.281045,0,0,0,0
1428,Team Pride,Unity Of Purpose,"""We're the best."" Instill a sense of elitism i...",The leader of the project must instill a comm

Ak je podobnosť viac ako 0.6 tak vzťah predpokladám že existuje

In [159]:
df_final.loc[df_final['cos_similarity_result_start_1']>0.6, 'result_start_1'] = 1
df_final.loc[df_final['cos_similarity_result_start_2']>0.6, 'result_start_2'] = 1
df_final.loc[df_final['cos_similarity_start_start']>0.6, 'start_start'] = 1
df_final.loc[df_final['cos_similarity_result_result']>0.6, 'result_result'] = 1
df_final

,pattern_name1,pattern_name2,resulting_context1,resulting_context2,starting_context1,starting_context2,starting_embedding1,resulting_embedding1,starting_embedding2,resulting_embedding2,cos_similarity_result_start_1,cos_similarity_start_start,cos_similarity_result_result,cos_similarity_result_start_2,result_start_1,result_start_2,start_start,result_result
0,Build Prototypes,Community Of Trust,Build an isolated prototype solution whose pur...,"Do things that explicitly demonstrate trust, s...",A project must test requirements and design de...,It is essential that the people in a team trus...,"[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.033992566, 0.0021839319, -0.0064396067, 0.0...","[0.013892247, 0.0677837, -0.011958739, 0.01426...",0.252579,0.399197,0.249860,0.350796,0,0,0,0
1,Build Prototypes,Completion Headroom,Build an isolated prototype solution whose pur...,Project work group completion dates from remai...,A project must test requirements and design de...,Every project must commit to delivery on a few...,"[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[-0.06432862, 0.046334974, -0.015850563, -0.08...","[-0.02881442, 0.105386145, -0.0058587473, -0.0...",0.147638,0.287624,0.218520,0.267191,0,0,0,0
2,Build Prototypes,Day Care,Build an isolated prototype solution whose pur...,"Put one expert in charge of all the novices, l...",A project must test requirements and design de...,Your experts are spending all their time mento...,"[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.069971845, 0.056567065, -0.00016744515, -0....","[0.071074046, -0.003414135, -0.03895503, 0.012...",0.224507,0.230168,0.334633,0.230830,0,0,0,0
3,Build Prototypes,Developer Controls Process,Build an isolated prototype solution whose pur...,Make the Developer the focal point of process ...,A project must test requirements and design de...,"A development culture, like any culture, can b...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.04957857, 0.05668401, -0.046187162, -0.0080...","[0.04166916, -0.056860633, -0.026997551, -0.03...",0.277095,0.514923,0.239971,0.340766,0,0,0,0
4,Build Prototypes,Development Episode,Build an isolated prototype solution whose pur...,Approach all development as a group activity a...,A project must test requirements and design de...,It's important to build on the collective stre...,"[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.0035809937, 0.0012670304, -0.019009784, 0.0...","[0.048495773, 0.012811256, -0.018782204, -0.00...",0.064959,0.175616,0.341376,0.349847,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1426,urrogate Customer,Unity Of Purpose,Create a SurrogateCustomer role in the project...,The leader of the project must instill a commo...,It is important to exchange ideas and clarify ...,Many projects have rocky beginnings as people ...,"[0.07129714, 0.05089531, -0.022737782, 0.02519...","[0.039613444, 0.06593218, -0.027706152, 0.0065...","[0.03618246, 0.029766077, -0.025872428, -1.552...","[0.05665192, 0.033690587, 0.00075172447, 0.032...",0.396681,0.392890,0.450400,0.375113,0,0,0,0
1427,urrogate Customer,Wise Fool,Create a SurrogateCustomer role in the project...,"Nurture the role of the wise fool, who can rai...",It is important to exchange ideas and clarify ...,Interpersonal dynamics often discourage good i...,"[0.07129714, 0.05089531, -0.022737782, 0.02519...","[0.039613444, 0.06593218, -0.027706152, 0.0065...","[0.03659603, 0.013783946, 0.027074216, 0.01551...","[0.04201864, 0.072648905, 0.027401034, 0.02183...",0.203926,0.287803,0.309779,0.281045,0,0,0,0
1428,Team Pride,Unity Of Purpose,"""We're the best."" Instill a sense of elitism i...",The leader of the project must instill a comm

## Vytváranie grafu pre 1. vzťah (konečný X a začiatočný Y)

In [163]:
values_1 = df_final[df_final['result_start_1']==1][['pattern_name1', 'pattern_name2']].values
values_2 = df_final[df_final['result_start_2']==1][['pattern_name1', 'pattern_name2']].values
pairs=[]
for value in values_1:
    pairs.append((value[0],value[1]))
for value in values_2:
    pairs.append((value[1],value[0]))
pairs

[('Community Of Trust', 'Developer Controls Process'),
 ('Completion Headroom', 'Work Split'),
 ('Development Episode', 'Unity Of Purpose'),
 ('Mercenary Analyst', 'Scenarios Define Problem'),
 ('Named Stable Bases', 'Private World'),
 ('Team Per Task', 'Holistic Diversity'),
 ('Team Per Task', 'Matron Role'),
 ('Diverse Groups', 'Self-Selecting Team'),
 ('EngageCustomers', 'Scenarios Define Problem'),
 ('EngageCustomers', 'urrogate Customer'),
 ('Engage Quality Assurance', 'Group Validation'),
 ('Work Flows Inward', 'Developer Controls Process'),
 ('Diverse Groups', 'Development Episode'),
 ('Domain Expertise In Roles', 'Apprenticeship'),
 ('urrogate Customer', 'EngageCustomers'),
 ('Group Validation', 'Engage Quality Assurance'),
 ('Unity Of Purpose', 'Team Pride')]

In [186]:
import networkx as nx
G = nx.DiGraph()
G.add_edges_from(pairs)

for node in G.nodes:
    language = df[df['pattern_names']==node]['pattern_language'].values
    if language == 'PiecemealGrowthPatternLanguage':
        G.nodes[node]['group']=0
    else:
        G.nodes[node]['group']=1

In [187]:
import pyvis
from pyvis.network import Network

net = Network(notebook=True, directed=True)
net.from_nx(G)

net.show("patterns.html")

patterns.html


In [190]:
net.save_graph("patterns.html")

## ML (nepoužité)

Jeden zo spôsobov ako použiť získané data s ML je napríklad použitie regresie, v tomto prípade som skúsil logistickú regresiu kde ako vstup idú metriky podbnosti. Samozrejme model potrebuje aj label-y takže som skúsil použiť moje predtým vytvorené informácie, či sú vzoru v postupnosti alebo nie. Ale nakoľko som použil len jednu sekvenciu (aj to možno nie s úplne správnym uvažovaním) model nedokázal na testovacej vzroke správne určiť 3 dvojice vzorov, ktoré som identifikoval ako páry v postupnosti

In [411]:
X = df_final[['cos_similarity','euc_distance']]
y = df_final['sequence']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=16)


In [412]:
logreg = LogisticRegression(random_state=16)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [414]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[160,   0],
       [  3,   0]], dtype=int64)

## Čo ďalej

V prvom rade rožšíriť databázu, spravovať aj ostatné jazkyky vzorov, prípadne ich príbehy o postupnosti (sekvencii). Taktiež nájsť riešnie ako použiť modely neurónovej siete na klasifikáciu párov textov (kontextov) na základe ich podobnosti.

Nájdenie thresholdu pre vzťah na základe podbností vektorov.

Pri prípade používania alternatívneho riešenia (text ako vektor). Navrhnutie metódy, ktorá bude schopná klasifikovať vzťahy vzorov na zákade ich vektorovej podobnosti - napríklad pomocou unsupervised learning (clustering)